# Project
*Team #1 - Connie Dunlap, Ryne Krueger, Jackson Robbins, Wyatt Tauber*

*CSEC-520/620 Cyber Analytics & Machine Learning*

## Prerequisites

This notebook requires the following packages:
* TBD

The following files and folder should be in the same directory as this notebook:
* TBD

## 1 Dataset Processing

TBD

In [12]:
"""
1 Dataset Processing
Author: Wyatt Tauber

TBD
"""
import csv
import subprocess

command = "rahash2 -a entropy"
string = "/home/wyatt/Desktop/csec620-project/UPX/1dd620a7293d700b1f15d48089a558aede814f53627dd87af2e77ca4fc6f94f5"
proc = subprocess.Popen([command + " " + string], stdout=subprocess.PIPE, shell=True)

(out, err) = proc.communicate()

print("program output:", out)

program output: b'/home/wyatt/Desktop/csec620-project/UPX/1dd620a7293d700b1f15d48089a558aede814f53627dd87af2e77ca4fc6f94f5: 0x00000000-0x0000e14d entropy: 3.90561958\n'
